In [3]:


from vdi.context_utils import enter_context
from vdi.db import db

@enter_context(lambda: db.connect())
async def f(c):
    return await c.fetch('select 1')

In [4]:

await db.init()
await f()

[<Record ?column?=1>]

In [11]:
async with db.connect() as c:
    print(await c.fetch('select 1'))

[<Record ?column?=1>]
